In [2]:
from javascript import require, On, Once, AsyncTask, once, off
import math

In [3]:
# Import the javascript libraries
mineflayer = require("mineflayer")
pathfinder = require('mineflayer-pathfinder')
vec3 = require("vec3")
# Global bot parameters
serverHost = "localhost"
serverPort = 3000
reconnect = True

In [4]:

class Agent:
    def __init__(self, playerUsername, botName, serverHost, serverPort, reconnect=True):
        self.playerUsername = playerUsername
        self.botName = botName
        self.serverHost = serverHost
        self.serverPort = serverPort
        self.reconnect = reconnect
        self.bot = None
        self.botArgs = {
            "host": serverHost,
            "port": serverPort,
            "username": botName,
            "hideErrors": False,
        }
        
        self.createBot()
        self.bot.loadPlugin(pathfinder.pathfinder)
        self.mcData = require('minecraft-data')(self.bot.version)
        self.movements = pathfinder.Movements(self.bot, self.mcData)

    def createBot(self):
        self.bot = mineflayer.createBot(self.botArgs)
        self.startEvents()
        return self.bot
    
    def log(self, message):
        print(f"[{self.bot.username}] {message}")

    def startEvents(self):
        bot = self.bot
        botName = self.botName
        reconnectState = {"reconnect": self.reconnect}

        @On(bot, "login")
        def handleLogin(this):
            botSocket = bot._client.socket
            print(f"[{botName}] Logged in to {botSocket.server if botSocket.server else botSocket._host }")

        @On(bot, "kicked")
        def handleKicked(this, reason, loggedIn):
            if loggedIn:
                print(f"[{botName}] Kicked whilst trying to connect: {reason}")

        @On(bot, "messagestr")
        def handleMessagestr(this, message, messagePosition, jsonMsg, sender, verified=None):
            if messagePosition == "chat" and "quit" in message:
                reconnectState["reconnect"] = False
                this.quit()
            elif messagePosition == "chat" and'come' in message:
                localPlayers = bot.players
                for player in localPlayers:
                        playerData = localPlayers[player]
                        if playerData["uuid"] == sender:
                            target = localPlayers[player].entity
                            
                if not target:
                    bot.chat("I don't see you !")
                    return
                pos = target.position
                bot.pathfinder.setMovements(self.movements)
                bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, 1))
            

        @On(bot, "end")
        def handleEnd(this, reason):
            print(f"[{botName}] Disconnected: {reason}")
            # Turn off old events
            off(bot, "login", handleLogin)
            off(bot, "kicked", handleKicked)
            off(bot, "messagestr", handleMessagestr)

            # Reconnect if the reconnect flag is set to True
            if reconnectState["reconnect"]:
                print(f"[{botName}] Attempting to reconnect")
                self.createBot()

            # Last event listener
            off(bot, "end", handleEnd)
        


    def setControlState(self, control, state):
        if self.bot:
            self.bot.setControlState(control, state)
        else:
            print("Bot is not created yet. Call createBot() first.")

    def move(self, direction, blocks):
        if direction in ["forward", "back", "left", "right"]:
            startPosition = self.bot.entity.position.clone()

            def calculateDistance(pos1, pos2):
                return math.sqrt((pos1.x - pos2.x) ** 2 + (pos1.z - pos2.z) ** 2+ (pos1.y - pos2.y) ** 2)

            def checkDistance(*args):
                currentPosition = self.bot.entity.position
                distance = calculateDistance(startPosition, currentPosition)
                if distance >= blocks:
                    self.bot.setControlState(direction, False)
                else:
                    self.bot.once('move', checkDistance)

            self.setControlState(direction, True)
            self.bot.once('move', checkDistance)
        else:
            print(f"Invalid move direction: {direction}")

    def forward(self, blocks):
        self.move("forward", blocks)

    def back(self, blocks):
        self.move("back", blocks)

    def left(self, blocks):
        self.move("left", blocks)

    def right(self, blocks):
        self.move("right", blocks)

    def jump(self, state=True):
        self.setControlState("jump", state)

    def stop(self):
        # Stops all movement
        self.setControlState("forward", False)
        self.setControlState("back", False)
        self.setControlState("left", False)
        self.setControlState("right", False)
        self.setControlState("jump", False)
    
    def pathfinding(self,goalPosition,findPlayer = None):
        if findPlayer == True:
            localPlayers = self.bot.players
            for player in localPlayers:
                    playerData = localPlayers[player]
                    if playerData["username"] == self.playerUsername:
                        target = localPlayers[player].entity
                        
            if not target:
                self.bot.chat("I don't see you !")
                return
            goalPosition = target.position
            self.bot.pathfinder.setMovements(self.movements)
            self.bot.pathfinder.setGoal(pathfinder.goals.GoalNear(goalPosition.x, goalPosition.y, goalPosition.z, 1))
        else:
            self.bot.pathfinder.setMovements(self.movements)
            self.bot.pathfinder.setGoal(pathfinder.goals.GoalNear(goalPosition.x, goalPosition.y, goalPosition.z, 1))
    
    def equip(self,itemString,destination):
         item = next((item for item in self.bot.inventory.items() if item.name == itemString), None)
         self.bot.equip(item, destination)
         

    def consume(self):
         self.bot.consume()
    
    def fish(self):
         self.bot.fish()
    
    def sleep(self,bedBlock):
         self.bot.sleep(bedBlock)
    
    def activateBlock(self,block): #  This is the same as right-clicking a block in the game. Useful for buttons, doors, etc. You must get to the block first
         self.bot.activateBlock(block)
    
    def lookAt(self,position): #  Look at the specified position. You must go near the position before you look at it. To fill bucket with water, you must lookAt first. `position` is `Vec3`
         self.bot.lookAt(position)
    
    def useOn(self,entity): # This is the same as right-clicking an entity in the game. Useful for shearing sheep, equipping harnesses, etc.
         self.bot.useOn(entity)

    def activateItem(self):
        self.bot.activateItem()



  





In [5]:
# agent = Agent('jessica030327',"Johnny", "localhost", 3000)
# agent1 = Agent('jessica030327',"Cassie", "localhost", 3000)
agent2 = Agent('jessica030327',"Jon", "localhost", 3000)


[Jon] Logged in to localhost


In [6]:
agent2.bot.inventory.items()

[
  Item {
    type: 15,
    count: 32,
    metadata: 0,
    nbt: null,
    stackId: null,
    name: 'dirt',
    displayName: 'Dirt',
    stackSize: 64,
    maxDurability: undefined,
    slot: 36
  }
]

In [8]:
agent2.bot.inventory.items()
agent2.equip('dirt','off-hand')

In [7]:
goalPosition = vec3(-201, 71, 500)
goalPosition
agent2.pathfinding(goalPosition,findPlayer = False)
agent1.pathfinding(goalPosition,findPlayer = False)
agent.pathfinding(goalPosition,findPlayer = False)

AttributeError: 'NoneType' object has no attribute 'setMovements'

In [7]:
# agent2.left(10)
# agent2.right(30)
agent2.forward(5)
# agent2.back(20)